In [0]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [0]:
dv_data = pd.read_csv("labeled4.csv")
new_dv = pd.read_csv("old_dv_30n31n32.csv")

In [3]:
dv_data.head(3)

,username,tweet_id,text,date,user_location,processed_text,lab_final
0,VipinHeartsTay,1.255927e+18,The fact their might be so many marital rape a...,2020-04-30 18:28:25+00:00,India,the fact their might be so many marital rape a...,DV_OPINION_ADVOCATE
1,TheGhoudi,1.255806e+18,"‘If the child has been through sexual abuse, a...",2020-04-30 10:26:35+00:00,"Mumbai, India",‘if the child ha be through sexual abuse adult...,NON_D_VIOLENCE_ABOUT
2,TheGhoudi,1.255803e+18,‘If you have any doubt & pick up any signs of ...,2020-04-30 10:14:19+00:00,"Mumbai, India",‘if you have any doubt pick up any sign of sex...,NON_D_VIOLENCE_ABOUT


In [4]:
new_dv.head(3)

,text,lab_final
0,Forced prostitution is a category of human tra...,DV_OPINION_INFO_NEWS
1,I visited Rudy one day I'd make comments and h...,DV_STORY
2,I have two for you -a guy creates a fighting t...,DV_OPINION_INFO_NEWS


In [0]:
dv_data = dv_data[['text','lab_final']]
final_df = pd.concat([dv_data,new_dv],axis=0)

In [6]:
final_df.shape

(3108, 2)

In [7]:
final_df['lab_final'].value_counts()

NON_D_VIOLENCE_ABOUT       1613
NO_VIOLENCE                 606
DV_OPINION_INFO_NEWS        294
NON_D_VIOLENCE_DIRECTED     274
DV_OPINION_ADVOCATE         170
DV_STORY                    130
DV_OPINION_DENIER            21
Name: lab_final, dtype: int64

In [0]:
dv_classes = pd.read_csv("data_dvclasses.csv")

In [0]:
dv_classes = dv_classes[['text','lab_final']]

In [0]:
final_df = pd.concat([final_df,dv_classes],axis=0)

In [0]:
final_df.reset_index(inplace=True)

In [0]:
final_df.dropna(inplace=True,axis=0)

In [0]:
final_df.drop(columns="index",axis=1,inplace=True)

In [14]:
final_df.shape

(3243, 2)

In [0]:
dv_29 = pd.read_csv("old_dv_29.csv")   #mainly for testing

In [0]:
final_df = pd.concat([final_df,dv_29],axis=0)

In [0]:
final_df.reset_index(inplace=True)
final_df.drop(columns="index",axis=1,inplace=True)

In [18]:
final_df['lab_final'].value_counts()

NON_D_VIOLENCE_ABOUT       1720
NO_VIOLENCE                 743
DV_OPINION_INFO_NEWS        398
NON_D_VIOLENCE_DIRECTED     310
DV_OPINION_ADVOCATE         296
DV_STORY                    204
DV_OPINION_DENIER            34
NON-D_VIOLENCE_ABOUT         33
DV_OPINIO _DENY               5
Name: lab_final, dtype: int64

In [0]:
final_df.replace("DV_OPINIO _DENY",value="DV_OPINION_DENIER",inplace=True)
final_df.replace("NON_D_VIOLENCE_DIRECTED",value="NON_D_VIOLENCE_ABOUT",inplace=True)
final_df.replace("NON-D_VIOLENCE_ABOUT",value="NON_D_VIOLENCE_ABOUT",inplace=True)
final_df.replace("DV_OPINION_INFO_NEWS",value="DV_OPINION",inplace=True)
final_df.replace("DV_OPINION_ADVOCATE",value="DV_OPINION",inplace=True)

In [0]:
label_encode = {}
for e,value in enumerate(final_df['lab_final'].unique()):
  label_encode[value] = e
final_df['label'] = final_df['lab_final'].map(label_encode)

In [21]:
label_encode

{'DV_OPINION': 0,
 'DV_OPINION_DENIER': 4,
 'DV_STORY': 3,
 'NON_D_VIOLENCE_ABOUT': 1,
 'NO_VIOLENCE': 2}

In [0]:
df_final = final_df.copy()

In [0]:
## We will go to the end to use the new method

## Traditional approach

In [24]:
df_final['label'].value_counts()

1    2063
2     743
0     694
3     204
4      39
Name: label, dtype: int64

In [25]:
import unicodedata  
from nltk import sent_tokenize
from nltk.corpus import stopwords
import nltk
import re
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        new_word = re.sub(r'(.)\1+', r'\1\1', new_word) 
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    stop_words = []
    stop_words.append(stopwords.words('english'))
    for word in words:
        if(word not in stop_words):
            new_words.append(word)
    return new_words

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

# def replace_contractions(text):
#     """Replace contractions in string of text"""
#     return contractions.fix(text)
def preprocess(text):
    # text = replace_contractions(text)
    content = nltk.word_tokenize(text)
    words = remove_non_ascii(content)
    words = to_lowercase(words)
    words = lemmatize_verbs(words)
    words = remove_punctuation(words)
    # words = remove_stopwords(words)
    # text= ' '.join(list(dict.fromkeys(words)))
    text = ' '.join(words)
    return text
def join_tags(words):
    text = ''.join(words)
    return text

In [0]:
df_final['text'] = df_final['text'].apply(preprocess)

In [28]:
df_final.head()

,text,lab_final,label
0,the fact their might be so many marital rape a...,DV_OPINION,0
1,if the child have be through sexual abuse adul...,NON_D_VIOLENCE_ABOUT,1
2,if you have any doubt pick up any sign of sexu...,NON_D_VIOLENCE_ABOUT,1
3,fuccbois a great film and yeah it talk about p...,NON_D_VIOLENCE_ABOUT,1
4,lockdown new time zone for sexual couple pictw...,NO_VIOLENCE,2


In [0]:
from sklearn.base import BaseEstimator, TransformerMixin

class TextSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on text columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None): # fit() doesn't do anything
        return self

    def transform(self, X):   # all the work is done here
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler

text = Pipeline([
                ('selector', TextSelector(key='text')),
                ('tfidf', TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}',ngram_range=(1,2)))
            ])

In [0]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion([('text', text)]) 
feature_processing = Pipeline([('feats', feats)])

In [32]:
df_final.head(2)

,text,lab_final,label
0,the fact their might be so many marital rape a...,DV_OPINION,0
1,if the child have be through sexual abuse adul...,NON_D_VIOLENCE_ABOUT,1


In [0]:
from sklearn.model_selection import train_test_split

In [0]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
x_train, x_valid, y_train, y_valid = train_test_split(df_final.drop(columns=['label'],axis=1), df_final['label'], 
                                                      random_state=0,test_size=0.15, stratify=df_final['label'])

In [0]:
x_train = feature_processing.fit_transform(x_train) 
x_valid = feature_processing.transform(x_valid)

In [34]:
from imblearn.over_sampling import SMOTE

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [36]:
sm = SMOTE()
x_train1, y_train1 = sm.fit_sample(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [0]:
unique_elements, counts_elements = np.unique(y_train1, return_counts=True)

In [38]:
unique_elements

array([0, 1, 2, 3, 4])

In [39]:
counts_elements

array([1753, 1753, 1753, 1753, 1753])

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [41]:
rf = RandomForestClassifier()
rf.fit(x_train1,y_train1)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
y_pred_smote = rf.predict(x_valid)

In [0]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

def evaluate_model(y_true, y_pred):
    print('confusion matrix')
    cm = confusion_matrix(y_true, y_pred)
    cm_df = pd.DataFrame(cm.T)
    cm_df.index.name = 'Predicted'
    cm_df.columns.name = 'True'
    print(cm_df)
    print('classification report')
    print(classification_report(y_true,y_pred))
    print('accuracy score')
    print(accuracy_score(y_true, y_pred))

In [44]:
evaluate_model(y_valid,y_pred_smote)

confusion matrix
True        0    1   2   3  4
Predicted                    
0          76    9   2   5  0
1          28  291  55  17  4
2           0   10  47   2  1
3           0    0   7   7  0
4           0    0   0   0  1
classification report
              precision    recall  f1-score   support

           0       0.83      0.73      0.78       104
           1       0.74      0.94      0.83       310
           2       0.78      0.42      0.55       111
           3       0.50      0.23      0.31        31
           4       1.00      0.17      0.29         6

    accuracy                           0.75       562
   macro avg       0.77      0.50      0.55       562
weighted avg       0.75      0.75      0.73       562

accuracy score
0.7508896797153025


In [45]:
label_encode

{'DV_OPINION': 0,
 'DV_OPINION_DENIER': 4,
 'DV_STORY': 3,
 'NON_D_VIOLENCE_ABOUT': 1,
 'NO_VIOLENCE': 2}

In [0]:
from sklearn.metrics import matthews_corrcoef
#matthews_corrcoef(y_valid,y_pred_smote)

In [0]:
from sklearn.svm import SVC

In [0]:
classifier = SVC(C = 3.5, kernel = 'linear', gamma='auto', random_state=42,class_weight="balanced")

In [49]:
classifier.fit(x_train1,y_train1)

SVC(C=3.5, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=42, shrinking=True, tol=0.001,
    verbose=False)

In [0]:
y_pred_smote_svc = classifier.predict(x_valid)

In [51]:
evaluate_model(y_valid,y_pred_smote_svc)

confusion matrix
True        0    1   2   3  4
Predicted                    
0          85   12   4   9  0
1          17  280  32   5  3
2           1   14  65   1  1
3           1    4  10  16  0
4           0    0   0   0  2
classification report
              precision    recall  f1-score   support

           0       0.77      0.82      0.79       104
           1       0.83      0.90      0.87       310
           2       0.79      0.59      0.67       111
           3       0.52      0.52      0.52        31
           4       1.00      0.33      0.50         6

    accuracy                           0.80       562
   macro avg       0.78      0.63      0.67       562
weighted avg       0.80      0.80      0.79       562

accuracy score
0.797153024911032


In [52]:
label_encode

{'DV_OPINION': 0,
 'DV_OPINION_DENIER': 4,
 'DV_STORY': 3,
 'NON_D_VIOLENCE_ABOUT': 1,
 'NO_VIOLENCE': 2}

In [0]:
from sklearn.ensemble import ExtraTreesClassifier 
from sklearn.linear_model import LogisticRegression
clf1 = SVC(C = 3.5, kernel = 'linear', gamma='auto', random_state=42,class_weight="balanced")
clf2 = RandomForestClassifier(random_state=88)
clf3 = SVC(C = 3.5, kernel = 'rbf', gamma='auto', random_state=42,class_weight="balanced")
lr = LogisticRegression()
#sclf = StackingClassifier(estimators=[clf1, clf2, clf3], final_estimator=lr)


In [41]:
!pip install vecstack
from vecstack import stacking 

  Created wheel for vecstack: filename=vecstack-0.4.0-cp36-none-any.whl size=19877 sha256=2cd1ae8f902f06093716b408c5cc3cf98d276b7891cef78a6a2e526ee2eab628
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [56]:
models = [clf1, clf2, lr]

# Compute stacking features
S_train, S_test = stacking(models, x_train1, y_train1, x_valid,
    regression = False, n_folds = 4,
    shuffle = True, random_state = 88, verbose = 2)

task:         [classification]
n_classes:    [5]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [SVC]
    fold  0:  [0.97627737]
    fold  1:  [0.97124601]
    fold  2:  [0.97489731]
    fold  3:  [0.96485623]
    ----
    MEAN:     [0.97181923] + [0.00442045]
    FULL:     [0.97181974]

model  1:     [RandomForestClassifier]
    fold  0:  [0.95483577]
    fold  1:  [0.94112277]
    fold  2:  [0.95253309]
    fold  3:  [0.94842538]
    ----
    MEAN:     [0.94922925] + [0.00521319]
    FULL:     [0.94922989]

model  2:     [LogisticRegression]
    fold  0:  [0.96167883]
    fold  1:  [0.94477408]
    fold  2:  [0.95207668]
    fold  3:  [0.94751255]
    ----
    MEAN:     [0.95151053] + [0.00642414]
    FULL:     [0.95151169]



In [57]:
S_train

array([[0, 0, 0],
       [1, 1, 2],
       [0, 0, 0],
       ...,
       [4, 4, 4],
       [4, 4, 4],
       [4, 4, 4]])

In [0]:
import xgboost

In [0]:

xgb = xgboost.XGBClassifier()
model = xgb

# Fit 2-nd level model
model = model.fit(S_train, y_train1)

# Predict
y_test_pred = model.predict(S_test)

In [60]:
evaluate_model(y_valid,y_test_pred)

confusion matrix
True        0    1   2   3  4
Predicted                    
0          89   16   5   7  0
1          13  273  31   6  3
2           1   18  65   0  1
3           1    3  10  18  0
4           0    0   0   0  2
classification report
              precision    recall  f1-score   support

           0       0.76      0.86      0.81       104
           1       0.84      0.88      0.86       310
           2       0.76      0.59      0.66       111
           3       0.56      0.58      0.57        31
           4       1.00      0.33      0.50         6

    accuracy                           0.80       562
   macro avg       0.79      0.65      0.68       562
weighted avg       0.80      0.80      0.79       562

accuracy score
0.7953736654804271


In [61]:
label_encode

{'DV_OPINION': 0,
 'DV_OPINION_DENIER': 4,
 'DV_STORY': 3,
 'NON_D_VIOLENCE_ABOUT': 1,
 'NO_VIOLENCE': 2}

In [46]:
!pip install py-thesaurus

  Created wheel for py-thesaurus: filename=py_thesaurus-1.0.5-cp36-none-any.whl size=6269 sha256=8444396c973276612d07ec06266f5035949e5abf1a78d66136caac46e1bcba0d
  Stored in directory: /root/.cache/pip/wheels/a6/c1/5a/124ea0abccb2ed21abcea99f376cd37d4dc99c8aa515a471f1
Successfully built py-thesaurus


In [0]:
from py_thesaurus import Thesaurus 
import nltk 
from nltk.corpus import wordnet 

In [44]:
!pip install nlpaug

     |████████████████████████████████| 102kB 7.2MB/s 


In [0]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

In [49]:
df_final['text'][0]

'the fact their might be so many marital rape and other sexual assault within home take place right now with a greater rate and no one wont be talk about it bcz corona'

In [50]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [51]:
aug = naw.SynonymAug(aug_src='wordnet')
augmented_text = aug.augment(df_final['text'][0])
print("Original:")
print(df_final['text'][0])
print("Augmented Text:")
print(augmented_text)

Original:
the fact their might be so many marital rape and other sexual assault within home take place right now with a greater rate and no one wont be talk about it bcz corona
Augmented Text:
the fact their might exist thus many marital rape and former sexual rape within home look at place right now with a greater rate and no one wont be spill the beans about information technology bcz corona


In [52]:
df_final.shape

(3743, 3)

In [53]:
import time
while 1:
  time.sleep(300)

KeyboardInterrupt: ignored

In [54]:
df_final.shape

(3743, 3)

In [0]:
df_final1 = df_final.iloc[0:3744]   

In [0]:
df_final_dv = df_final1[(df_final1['label'] == 3) | (df_final1['label'] ==4)]

In [57]:
df_final_dv.head(2)

,text,lab_final,label
11,cat valentinecat be so aloof because she be su...,DV_STORY,3
179,he have have vulgar clip i ll produce the snap...,DV_STORY,3


In [0]:
def augmentation(text):
  aug = naw.SynonymAug(aug_src='wordnet')
  augmented_text = aug.augment(text)
  return augmented_text

In [59]:
df_final_dv['text_syn'] = df_final_dv['text'].apply(augmentation)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [60]:
df_final_dv_syn = df_final_dv[['text_syn','label']]
df_final_dv_syn.rename(columns={'text_syn':'text'},inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
df_final1 = pd.concat([df_final1,df_final_dv_syn],axis=0)
df_final1.reset_index(inplace=True)
df_final1.drop(columns="index",axis=1,inplace=True)
print(df_final1.shape)
df_final = pd.concat([df_final1,df_final.iloc[3243:]],axis=0)
df_final.reset_index(inplace=True)
df_final.drop(columns="index",axis=1,inplace=True)

(3410, 3)


In [0]:
df_final = pd.concat([df_final,df_final_dv_syn],axis=0)

In [121]:
inference_data = pd.read_csv("inference_data.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [63]:
inference_data.head(2)

,date,favorites,formatted_date,hashtags,mentions,permalink,replies,retweets,text,to,tweet_id,tweet_location,urls,user_id,user_location,username
0,2020-01-01 00:00:24+00:00,5,Wed Jan 01 00:00:24 +0000 2020,NaN,NaN,https://twitter.com/Marielmallarii/status/1212...,0,0,2020 please dont let me hurt again pic.twitter...,NaN,1212161612913639424,NaN,NaN,1190441837489639424,i'm lost,Marielmallarii
1,2020-01-01 00:04:09+00:00,25,Wed Jan 01 00:04:09 +0000 2020,#SudhirLostThePoll,@sudhirchaudhary,https://twitter.com/Akashtv1Soni/status/121216...,16,12,His detractors jumped the gun. The result of @...,NaN,1212162559438213125,NaN,NaN,932231099060690945,NaN,Akashtv1Soni


In [0]:
inference_data = inference_data[['text']]

In [0]:
df_final1 = pd.concat([df_final,inference_data],axis=0)

In [126]:
df_final1.isnull().sum()

text                   6
lab_final         264923
label             264680
date                3986
favorites           3986
formatted_date      3986
hashtags          215980
mentions          234252
permalink           3986
replies             3986
retweets            3986
to                160552
tweet_id            3986
tweet_location    268666
urls              214899
user_id             3986
user_location     198768
username            3986
dtype: int64

In [127]:
df_final1.dropna(axis=0,subset=['text'],inplace=True)
df_final1.isnull().sum()

text                   0
lab_final         264917
label             264674
date                3986
favorites           3986
formatted_date      3986
hashtags          215974
mentions          234246
permalink           3986
replies             3986
retweets            3986
to                160552
tweet_id            3986
tweet_location    268660
urls              214899
user_id             3986
user_location     198763
username            3986
dtype: int64

In [128]:
df_final1.shape

(268660, 18)

In [0]:
df_final.reset_index(inplace=True)
df_final.drop("index",axis=1,inplace=True)

In [0]:
df_final1.drop_duplicates(subset=['text'], keep="first",inplace=True)

In [131]:
df_final1.shape  

(262232, 18)

In [132]:
df_final1.iloc[3904:].shape

(258328, 18)

In [71]:
df_final1.iloc[:3903].isnull().sum()   #going to be our training set

text           0
lab_final    167
label          0
dtype: int64

In [0]:
df_final1['text'] = df_final1['text'].apply(preprocess)

In [0]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion([('text', text)]) 
feature_processing = Pipeline([('feats', feats)])

In [74]:
df_final1.head(2)

,text,lab_final,label
0,the fact their might be so many marital rape a...,DV_OPINION,0.0
1,if the child have be through sexual abuse adul...,NON_D_VIOLENCE_ABOUT,1.0


In [75]:
df_final1.shape

(262232, 3)

In [0]:
x_train = df_final.drop(columns=['label','lab_final'],axis=1).iloc[:3903]
y_train = df_final['label'].iloc[:3903]

x_test = df_final1.drop(columns=['label','lab_final'],axis=1).iloc[3904:]  # inference dataset
#x_valid = df_final.drop(columns=['label','lab_final'],axis=1).iloc[3410:]
#y_valid = df_final['label'].iloc[3410:]

In [0]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
x_train, x_valid, y_train, y_valid = train_test_split(df_final.drop(columns=['label','lab_final'],axis=1), df_final['label'], 
                                                      random_state=0,test_size=0.15, stratify=df_final['label'])

In [0]:
x_train = feature_processing.fit_transform(x_train) 
x_test = feature_processing.transform(x_test)

In [0]:
from imblearn.over_sampling import SMOTE

In [80]:
y_train

0       0
1       1
2       1
3       1
4       2
       ..
3139    3
3140    3
3153    3
3169    4
3170    3
Name: label, Length: 3903, dtype: int64

In [85]:
sm = SMOTE()
x_train1, y_train1 = sm.fit_sample(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [0]:
unique_elements, counts_elements = np.unique(y_train1, return_counts=True)

In [87]:
unique_elements

array([0, 1, 2, 3, 4])

In [88]:
counts_elements

array([2063, 2063, 2063, 2063, 2063])

In [89]:
x_test.shape

(258328, 82460)

In [0]:
from sklearn.ensemble import ExtraTreesClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
clf1 = SVC(C = 3.5, kernel = 'linear', gamma='auto', random_state=42,class_weight="balanced")
clf2 = RandomForestClassifier(random_state=88)
clf3 = SVC(C = 3.5, kernel = 'rbf', gamma='auto', random_state=42,class_weight="balanced")
lr = LogisticRegression()



In [0]:
#!pip install vecstack
from vecstack import stacking 

In [92]:
models = [clf1, clf2, lr]

# Compute stacking features
S_train, S_test = stacking(models, x_train1, y_train1, x_test,
    regression = False, n_folds = 4,
    shuffle = True, random_state = 88, verbose = 2)

task:         [classification]
n_classes:    [5]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [SVC]
    fold  0:  [0.96936797]
    fold  1:  [0.97053121]
    fold  2:  [0.96393951]
    fold  3:  [0.97478666]
    ----
    MEAN:     [0.96965634] + [0.00386817]
    FULL:     [0.96965584]

model  1:     [RandomForestClassifier]
    fold  0:  [0.95191935]
    fold  1:  [0.94804188]
    fold  2:  [0.93951144]
    fold  3:  [0.95034911]
    ----
    MEAN:     [0.94745544] + [0.00478933]
    FULL:     [0.94745516]

model  2:     [LogisticRegression]
    fold  0:  [0.95036836]


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


    fold  1:  [0.94377666]


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


    fold  2:  [0.94067468]
    fold  3:  [0.95577967]
    ----
    MEAN:     [0.94764984] + [0.00585532]
    FULL:     [0.94764905]



In [0]:
import xgboost
xgb = xgboost.XGBClassifier()
model = xgb

# Fit 2-nd level model
model = model.fit(S_train, y_train1)

# Predict
y_test_pred = model.predict(S_test)

In [94]:
y_test_pred.shape

(258328,)

In [0]:
predicted = pd.DataFrame({'text':x_test,'label':y_test_pred})

In [96]:
predicted.head(2)

,text,label
0,"(0, 48)\t0.09776125125283194\n (0, 802)\t0....",3
1,"(0, 48)\t0.09776125125283194\n (0, 802)\t0....",3


In [123]:
df_final1.shape

(268666, 18)

In [109]:
predicted.shape

(258328, 2)

In [0]:
final_inference = df_final1.iloc[3904:]

In [134]:
final_inference.shape

(258328, 18)

In [135]:
final_inference.reset_index(inplace=True)
final_inference.drop("index",axis=1,inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
final_inference = pd.concat([final_inference,predicted['label']],axis=1)

In [138]:
final_inference.head(10)

,text,lab_final,label,date,favorites,formatted_date,hashtags,mentions,permalink,replies,retweets,to,tweet_id,tweet_location,urls,user_id,user_location,username,label
0,nah fam 1 same baby boy but information techno...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1,yanno i ve always wonder something now hear ma...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
2,a house of smoke and mirror shortly after her ...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
3,while i don t condone violenceproperty damage ...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,think a man try rape his step daughter try to ...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
5,hi house can a husband rape his wife i need ho...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
6,why a man canful not despoil his lawfully wed ...,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
7,the law be make nothing gevatsecrhe first pers...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
8,help get strangulation a felony and measure pa...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
9,first sour my female parent have no right hand...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3


In [143]:
final_inference.columns

Index(['text', 'label', 'date', 'favorites', 'formatted_date', 'hashtags',
       'mentions', 'permalink', 'replies', 'retweets', 'to', 'tweet_id',
       'tweet_location', 'urls', 'user_id', 'user_location', 'username',
       'label'],
      dtype='object')

In [0]:
final_inference.columns = ['text', 'label1', 'date', 'favorites', 'formatted_date', 'hashtags',
       'mentions', 'permalink', 'replies', 'retweets', 'to', 'tweet_id',
       'tweet_location', 'urls', 'user_id', 'user_location', 'username',
       'label']

In [0]:
final_inference.drop(columns=['label1'],axis=1,inplace=True)

In [148]:
final_inference['label'].value_counts()

1    149859
2     76046
0     18231
3     14166
4        26
Name: label, dtype: int64

In [149]:
label_encode

{'DV_OPINION': 0,
 'DV_OPINION_DENIER': 4,
 'DV_STORY': 3,
 'NON_D_VIOLENCE_ABOUT': 1,
 'NO_VIOLENCE': 2}

In [0]:
label_name = {y:x for x,y in label_encode.items()}

In [152]:
label_name

{0: 'DV_OPINION',
 1: 'NON_D_VIOLENCE_ABOUT',
 2: 'NO_VIOLENCE',
 3: 'DV_STORY',
 4: 'DV_OPINION_DENIER'}

In [0]:
final_inference['lab_final'] = final_inference['label'].map(label_name)

In [157]:
final_inference['lab_final'].value_counts()

NON_D_VIOLENCE_ABOUT    149859
NO_VIOLENCE              76046
DV_OPINION               18231
DV_STORY                 14166
DV_OPINION_DENIER           26
Name: lab_final, dtype: int64

In [168]:
final_inference.iloc[5000]

text                      Super hit bhojpuri song khesari bhaiya ji
date                                      2020-01-04 05:00:50+00:00
favorites                                                         2
formatted_date                       Sat Jan 04 05:00:50 +0000 2020
hashtags                                                        NaN
mentions                                                        NaN
permalink         https://twitter.com/Actorsinger17/status/12133...
replies                                                           0
retweets                                                          0
to                                                        khesariLY
tweet_id                                                1.21332e+18
tweet_location                                                  NaN
urls                                                            NaN
user_id                                                  1.2059e+18
user_location                                   

In [0]:
final_inference.to_csv("inference_data_with_label.csv")

In [107]:
evaluate_model(y_valid,y_test_pred)

confusion matrix
True        0    1   2   3  4
Predicted                    
0          81   14   8   5  0
1          16  277  36   2  1
2           3   15  64   1  0
3           4    3   3  44  0
4           0    0   0   0  9
classification report
              precision    recall  f1-score   support

           0       0.75      0.78      0.76       104
           1       0.83      0.90      0.86       309
           2       0.77      0.58      0.66       111
           3       0.81      0.85      0.83        52
           4       1.00      0.90      0.95        10

    accuracy                           0.81       586
   macro avg       0.83      0.80      0.81       586
weighted avg       0.81      0.81      0.81       586

accuracy score
0.810580204778157


In [0]:
label_encode

{'DV_OPINION': 0,
 'DV_OPINION_DENIER': 4,
 'DV_STORY': 3,
 'NON_D_VIOLENCE_ABOUT': 1,
 'NO_VIOLENCE': 2}

In [0]:
matthews_corrcoef(y_valid,y_test_pred)

0.7854014758796973